<a href="https://colab.research.google.com/github/pexize/Cria-o-de-Uma-Base-de-Dados-e-Treinamento-da-Rede-YOLO/blob/main/rede%20yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Configurar o Ambiente
# Não precisamos instalar o TensorFlow, pois ele já está instalado no Colab.
!git clone https://github.com/zzh8829/yolov3-tf2.git
%cd yolov3-tf2

# Remover a linha do TensorFlow do requirements.txt
!sed -i '/tensorflow/d' requirements.txt
!sed -i 's/opencv-python==4.2.0.32/opencv-python/' requirements.txt

fatal: destination path 'yolov3-tf2' already exists and is not an empty directory.
/content/yolov3-tf2


In [ ]:
# Instalar as dependências restantes
!pip install -r requirements.txt


Obtaining file:///content/yolov3-tf2 (from -r requirements.txt (line 5))
  Preparing metadata (setup.py) ... done
  Attempting uninstall: yolov3_tf2
    Found existing installation: yolov3_tf2 0.1
    Uninstalling yolov3_tf2-0.1:
      Successfully uninstalled yolov3_tf2-0.1
  Running setup.py develop for yolov3_tf2


In [ ]:
import tensorflow as tf
from absl import flags
from absl.flags import FLAGS
import os
import xml.etree.ElementTree as ET

In [ ]:
# 2. Baixar um Dataset
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!tar -xvf VOCtrainval_06-Nov-2007.tar

In [ ]:
!mkdir dataset
!mv VOCdevkit/VOC2007/JPEGImages/* dataset/images
!mv VOCdevkit/VOC2007/Annotations/* dataset/labels

mkdir: cannot create directory ‘dataset’: File exists


In [ ]:
# Verificar se há imagens no diretório
image_files = !ls dataset/images
if not image_files:
    raise FileNotFoundError("Nenhuma imagem encontrada em 'dataset/images'. Verifique o download e a organização dos dados.")


In [ ]:
# Limitar o dataset a apenas algumas imagens para economizar espaço
!ls dataset/images | head -n 50 > dataset/train.txt
!ls dataset/images | tail -n 10 > dataset/val.txt

In [ ]:
 #Verificar se os arquivos train.txt e val.txt foram criados
if not os.path.exists('dataset/train.txt') or not os.path.exists('dataset/val.txt'):
    raise FileNotFoundError("Os arquivos 'train.txt' ou 'val.txt' não foram criados. Verifique o diretório 'dataset/images'.")

In [ ]:
# 3. Preparar os Dados
# Função para converter XML para TFRecord
def create_tf_example(image_path, annotation_path, class_map):
    # Ler a imagem
    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_jpg = fid.read()

    # Decodificar a imagem para obter as dimensões
    image = tf.image.decode_image(encoded_jpg, channels=3)
    image_shape = tf.shape(image)  # Obter as dimensões da imagem
    height = image_shape[0].numpy()  # Altura
    width = image_shape[1].numpy()   # Largura

    # Ler o arquivo XML
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    # Extrair informações das anotações
    xmin = []
    xmax = []
    ymin = []
    ymax = []
    class_texts = []  # Nomes das classes (strings codificadas como bytes)
    class_indices = []  # Índices das classes (inteiros)
    for obj in root.findall('object'):
        label = obj.find('name').text
        if label not in class_map:
            continue
        class_texts.append(label.encode('utf8'))  # Nome da classe como bytes
        class_indices.append(class_map[label])    # Índice da classe
        bbox = obj.find('bndbox')
        xmin.append(float(bbox.find('xmin').text) / width)
        xmax.append(float(bbox.find('xmax').text) / width)
        ymin.append(float(bbox.find('ymin').text) / height)
        ymax.append(float(bbox.find('ymax').text) / height)

    # Criar o exemplo TFRecord
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[os.path.basename(image_path).encode('utf8')])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[os.path.basename(image_path).encode('utf8')])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_jpg])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=['jpeg'.encode('utf8')])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmin)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmax)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymin)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymax)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=class_texts)),  # Nomes das classes
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=class_indices)),  # Índices das classes
    }))
    return tf_example

In [ ]:
def convert_to_tfrecord(data_dir, output_dir, split='train'):
    # Mapeamento de classes
    class_map = {name: idx + 1 for idx, name in enumerate(['aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
                                                           'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
                                                           'dog', 'horse', 'motorbike', 'person', 'pottedplant',
                                                           'sheep', 'sofa', 'train', 'tvmonitor'])}

    # Ler lista de imagens
    split_file = os.path.join(data_dir, f'{split}.txt')
    with open(split_file, 'r') as f:
        image_names = [line.strip().split('.')[0] for line in f.readlines()]

    # Criar arquivo TFRecord
    output_path = os.path.join(output_dir, f'{split}.tfrecord')
    with tf.io.TFRecordWriter(output_path) as writer:
        for image_name in image_names:
            image_path = os.path.join(data_dir, 'images', f'{image_name}.jpg')
            annotation_path = os.path.join(data_dir, 'labels', f'{image_name}.xml')
            if not os.path.exists(annotation_path):
                print(f"Anotação ausente para a imagem: {image_name}")
                continue
            tf_example = create_tf_example(image_path, annotation_path, class_map)
            writer.write(tf_example.SerializeToString())


In [ ]:
# Converter dados para TFRecord
convert_to_tfrecord('dataset', 'data', split='train')
convert_to_tfrecord('dataset', 'data', split='val')
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [36]:
# 4. Baixar Pesos Pré-treinados
# Baixar pesos pré-treinados para inicialização
!wget https://pjreddie.com/media/files/yolov3.weights
!mv yolov3.weights ./data/
!python convert.py --weights ./data/yolov3.weights --output ./checkpoints/yolov3.weights.h5
print("Preparação concluída! Os dados foram convertidos para TFRecord e os pesos pré-treinados foram baixados.")

--2025-02-06 02:23:14--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  40.0MB/s    in 6.4s    

2025-02-06 02:23:20 (36.9 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

2025-02-06 02:23:21.783816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738808601.813863   36526 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738808601.821841   36526 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory fo

In [35]:
!mv yolov3.weights ./data/

mv: cannot stat 'yolov3.weights': No such file or directory


In [44]:
!python train.py \
    --batch_size 4 \
    --dataset data/train.tfrecord \
    --val_dataset data/val.tfrecord \
    --epochs 10 \
    --mode fit \
    --transfer fine_tune \
    --weights ./checkpoints/yolov3.weights.h5 \
    --num_classes 20 \
    --classes data/voc2007.names

2025-02-06 02:37:55.655035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738809475.692645   40061 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738809475.701150   40061 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 02:38:02.447806: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
Epoch 1/10
Instructions for updating:
Use fn_output_signature instead
W0206 